In [32]:
pip install tensorflow_datasets

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 4.3 MB 12.6 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 5.0 MB/s s eta 0:00:01
     |████████████████████████████████| 124 kB 138.7 MB/s eta 0:00:01
     |████████████████████████████████| 211 kB 125.5 MB/s eta 0:00:01
  Created wheel for promise: filename=promise-2.3-py3-none-any.whl size=21486 sha256=df110a1d471c8eef1c81a79a83887a169daf83d5f90c0c387151d1893a43b0e8
  Stored in directory: /p/home/jusers/langguth1/juwels/.cache/pip/wheels/e1/e8/83/ddea66100678d139b14bc87692ece57c6a2a937956d2532608
Successfully built promise
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os, sys
sys.path.append("../handle_data/")
from collections import OrderedDict
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import LearningRateScheduler

# all the layers used for U-net
from tensorflow.keras.layers import (Activation, BatchNormalization, Concatenate, Conv2D,
                                     Conv2DTranspose, Input, MaxPool2D, Dense, Flatten, GlobalAveragePooling2D
)
from tensorflow.keras.models import Model
# other modules
import gc
import xarray as xr
import numpy as np


from handle_data_unet import HandleUnetData

In [40]:
def split_in_tar(da: xr.DataArray, target_var: str = "t2m") -> (xr.DataArray, xr.DataArray):
    
    invars = [var for var in da["variables"].values if var.endswith("_in")]
    tarvars = [var for var in da["variables"].values if var.endswith("_tar")]

    # ensure that ds_tar has a channel coordinate even in case of single target variable
    roll = False
    if len(tarvars) == 1:
        sl_tarvars = tarvars
    else:
        sl_tarvars = slice(*tarvars)
        if tarvars[0] != target_var:     # ensure that target variable appears as first channel
            roll = True

    da_in, da_tar = da.sel({"variables": invars}), da.sel(variables=sl_tarvars)
    if roll: da_tar = da_tar.roll(variables=1, roll_coords=True)

    return da_in, da_tar


def make_data_generator(da: xr.DataArray, da_val: xr.DataArray = None, embed=None, embed_val=None,
                        var2drop: str = "z_tar", nepochs: int = 1) -> tf.data.Dataset:

    if not True:
        da = da.drop(var2drop, dim="variables")

    da = da.load()
    da_in, da_tar = split_in_tar(da)

    def gen(darr_in, darr_tar):
        # darr_in, darr_tar = darr_in.load(), darr_tar.load()
        ntimes = len(darr_in["time"])
        for t in range(ntimes):
            yield tuple((darr_in.isel({"time": t}).values, darr_tar.isel({"time": t}).values))

    s0 = next(iter(gen(da_in, da_tar)))
    sample_spec = {"shape_in": s0[0].shape, "type_in": s0[0].dtype , 
                   "shape_tar": s0[1].shape, "type_tar": s0[0].dtype}

    gen_train = gen(da_in, da_tar)

    if False:
        if not embed:
            raise ValueError("Embedding is enabled, but no embedding data was parsed.")
        data_iter = tf.data.Dataset.from_tensor_slices((ds_in, ds_tar, embed))
    else:
        #data_iter = tf.data.Dataset.from_tensor_slices((ds_in, ds_tar))
        data_iter = tf.data.Dataset.from_generator(lambda: gen_train, 
                                                       output_signature=(tf.TensorSpec(sample_spec["shape_in"],
                                                                                       dtype=sample_spec["type_in"]),
                                                                         tf.TensorSpec(sample_spec["shape_tar"],
                                                                                       dtype=sample_spec["type_tar"])))

    # Notes:
    # * cache is reuqired to make repeat work properly on datasets based on generators
    #   (see https://stackoverflow.com/questions/60226022/tf-data-generator-keras-repeat-does-not-work-why)
    # * repeat must be applied after shuffle to get varying mini-batches per epoch
    # * batch-size is increaded to allow substepping in train_step
    data_iter = data_iter.cache().shuffle(20000).batch(32 * (6 + 1), drop_remainder=True).repeat()
    del da
    gc.collect()
    # data_iter = data_iter.prefetch(tf.data.AUTOTUNE)


    if da_val is not None:
        da_val =da_val.load()
        ds_val_in, ds_val_tar = split_in_tar(da_val)
        gen_val = gen(da_val_in, da_val_tar)

        if False:
            if not embed_val:
                raise ValueError("Embedding is enabled, but no embedding data for validation dataset is parsed.")
            val_iter = tf.data.Dataset.from_tensor_slices((ds_val_in, ds_val_tar, embed_val))
        else:
            # val_iter = tf.data.Dataset.from_tensor_slices((ds_val_in, ds_val_tar))
            val_iter = tf.data.Dataset.from_generator(lambda: gen_val, 
                                                      output_signature=(tf.TensorSpec(sample_spec["shape_in"],
                                                                                      dtype=sample_spec["type_in"]),
                                                                        tf.TensorSpec(sample_spec["shape_tar"],
                                                                                      dtype=sample_spec["type_tar"])))

        val_iter = val_iter.cache().batch(32, drop_remainder=True).repeat()

        return data_iter, val_iter
    else:
        return data_iter

    
# preprocess data (i.e. normalizing)
def reshape_ds(ds):
    da = ds.to_array(dim="variables")
    da = da.transpose(..., "variables")
    return da


In [18]:
datadir = "/p/scratch/deepacf/maelstrom/maelstrom_data/ap5_michael/preprocessed_era5_crea6/netcdf_data/all_files/"

In [8]:
ds_train = xr.open_dataset(os.path.join(datadir, "preproc_era5_crea6_train.nc"))

da_train = reshape_ds(ds_train)

norm_dims = ["time", "rlat", "rlon"]
da_train, mu_train, std_train = HandleUnetData.z_norm_data(da_train, dims=norm_dims, return_stat=True)

del ds_train

In [9]:
gc.collect()

da_train = da_train.astype(np.float32)

In [41]:
import tensorflow_datasets as tfd

tfds = make_data_generator(da_train, nepochs=10)

2022-09-05 16:14:32.061587: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [22]:
from tqdm import tqdm

In [39]:
print(np.shape(da_train))

(94052, 96, 120, 12)


In [42]:
tfds_i = iter(tfds)
ntakes = 1000
sh_ok = (32, 96, 120,9)

for i in tqdm(range(ntakes)):
    print(f"Train step: {i:d}")
    print("Samples: {0:d} : {1:d}".format(i*224, (i+1)*224))
    batch_in, batch_tar = tfds_i.get_next()
    for j in range(7):
        ist, ie = j * 32, (j + 1) * 32
        assert np.shape(batch_in[ist:ie, ...]) == sh_ok, "Incomplete batch. Expected: ({0}), got: ({1})".format(",".join(str(x) for x in sh_ok), 
                                                                                                                ",".join(str(x) for x in np.shape(batch_in[ist:ie,...])))


  0%|          | 0/1000 [00:00<?, ?it/s]

Train step: 0
Samples: 0 : 224


2022-09-05 16:16:29.422291: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:175] Filling up shuffle buffer (this may take a while): 14726 of 20000
2022-09-05 16:16:33.025989: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:228] Shuffle buffer filled.
  0%|          | 2/1000 [00:13<1:36:21,  5.79s/it]

Train step: 1
Samples: 224 : 448
Train step: 2
Samples: 448 : 672


  0%|          | 4/1000 [00:14<33:47,  2.04s/it]  

...

Train step: 983
Samples: 220192 : 220416
Train step: 984
Samples: 220416 : 220640
Train step: 985
Samples: 220640 : 220864
Train step: 986
Samples: 220864 : 221088
Train step: 987
Samples: 221088 : 221312
Train step: 988
Samples: 221312 : 221536
Train step: 989
Samples: 221536 : 221760


100%|█████████▉| 995/1000 [01:32<00:00, 34.11it/s]

Train step: 990
Samples: 221760 : 221984
Train step: 991
Samples: 221984 : 222208
Train step: 992
Samples: 222208 : 222432
Train step: 993
Samples: 222432 : 222656
Train step: 994
Samples: 222656 : 222880
Train step: 995
Samples: 222880 : 223104
Train step: 996
Samples: 223104 : 223328


100%|██████████| 1000/1000 [01:33<00:00, 10.75it/s]

Train step: 997
Samples: 223328 : 223552
Train step: 998
Samples: 223552 : 223776
Train step: 999
Samples: 223776 : 224000


In [43]:
print(156*7*32)

34944


In [52]:
#len(list(tfds))

tfds.cardinality().numpy()

-2

In [66]:
# 2022-09-16: Test embedding implementation for Keras model

In [66]:
from tensorflow.keras.layers import (Concatenate, Input, Dense, CategoryEncoding, Reshape, MaxPool2D, RepeatVector)
from tensorflow.keras.models import Model
import tensorflow as tf
import numpy as np

In [27]:
input_shape = (120, 96, 9)
embed_shape = [[24], [12]]

l_embed = True

In [79]:
if l_embed:
    n_nodes = np.prod(input_shape[:-1])
    tar_dim = tuple(list(input_shape[:-1]) + [1])
    
    embed_vec_in = [Input(shape=i) for i in embed_shape]
    
    # project embedding to scalar value
    embed_vec_proj = [Dense(1)(i) for i in embed_vec_in]
    # repeat and reshape to match input-data for concatenation
    embed_vec_proj = [RepeatVector(int(n_nodes))(i) for i in embed_vec_proj]
    embed_vec_proj = [Reshape(tar_dim)(i) for i in embed_vec_proj]
    
inputs = Input(input_shape)    

if l_embed:
    merge_list = [inputs] + [embed for embed in embed_vec_proj]
    
    merge = Concatenate()(merge_list)
    #merge = Concatenate()([inputs, li_h, li_m])
else:
    print("No concat")
    merge = inputs

out = MaxPool2D((2,2))(merge)

[<KerasTensor: shape=(None, 24) dtype=float32 (created by layer 'input_85')>, <KerasTensor: shape=(None, 12) dtype=float32 (created by layer 'input_86')>]
11520


In [80]:
#model = Model(inputs, out, name="t2m_downscaling_unet")
all_inputs = [inputs] + [input_embed for input_embed in embed_vec_in]
model = Model(all_inputs, out, name="t2m_downscaling_unet")
#model = Model([inputs, in_embed_hour, in_embed_month], out, name="t2m_downscaling_unet")

In [81]:
print(model.summary())

Model: "t2m_downscaling_unet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_85 (InputLayer)           [(None, 24)]         0                                            
__________________________________________________________________________________________________
input_86 (InputLayer)           [(None, 12)]         0                                            
__________________________________________________________________________________________________
dense_65 (Dense)                (None, 1)            25          input_85[0][0]                   
__________________________________________________________________________________________________
dense_66 (Dense)                (None, 1)            13          input_86[0][0]                   
_______________________________________________________________________________

# 2022-09-16: Test creation of TF dataset with embedding

In [92]:
import os
import xarray as xr
import tensorflow as tf
from tensorflow.keras.utils import to_categorical as OneHotEncoder
import pandas as pd

def reshape_ds(ds):
    """
    Convert a xarray dataset to a data-array where the variables will constitute the last dimension (channel last)
    :param ds: the xarray dataset with dimensions (dims)
    :return da: the data-array with dimensions (dims, variables)
    """
    da = ds.to_array(dim="variables")
    da = da.transpose(..., "variables")
    return da

def split_in_tar(da: xr.DataArray, target_var: str = "t2m") -> (xr.DataArray, xr.DataArray):
    """
    Split data array with variables-dimension into input and target data for downscaling.
    :param da: The unsplitted data array.
    :param target_var: Name of target variable which should consttute the first channel
    :return: The splitted data array.
    """
    invars = [var for var in da["variables"].values if var.endswith("_in")]
    tarvars = [var for var in da["variables"].values if var.endswith("_tar")]

    # ensure that ds_tar has a channel coordinate even in case of single target variable
    roll = False
    if len(tarvars) == 1:
        sl_tarvars = tarvars
    else:
        sl_tarvars = slice(*tarvars)
        if tarvars[0] != target_var:     # ensure that target variable appears as first channel
            roll = True

    da_in, da_tar = da.sel({"variables": invars}), da.sel(variables=sl_tarvars)
    if roll: da_tar = da_tar.roll(variables=1, roll_coords=True)

    return da_in, da_tar

In [102]:
datadir = "/p/scratch/deepacf/maelstrom/maelstrom_data/ap5_michael/preprocessed_era5_crea6/netcdf_data/all_files"
nc_fname = "preproc_era5_crea6_val.nc"

nc_file = os.path.join(datadir, nc_fname)

ds = xr.open_dataset(nc_file)
da = reshape_ds(ds)

In [117]:
def make_tf_dataset(da: xr.DataArray, batch_size: int = 32, lshuffle: bool = True, shuffle_samples: int = 20000,
                    named_targets: bool = False, var_tar2in: str = None,
                    lembed_date: bool = True) -> tf.data.Dataset:

    da = da.load()
    da_in, da_tar = split_in_tar(da)
    if var_tar2in is not None:
        da_in = xr.concat([da_in, da_tar.sel({"variables": var_tar2in})], "variables")

    varnames_tar = da_tar["variables"].values
    
    def gen_named(darr_in, darr_tar, lt_embed: bool = False):
        """
        Generator with named output streams (as dictionary).
        """            
        ntimes = len(darr_in["time"])
        for t in range(ntimes):
            tar_now = darr_tar.isel({"time": t})
            time = darr_in.isel({"time": t})["time"].values
            all_streams = [darr_in.isel({"time": t}).values, 
                           {var: tar_now.sel({"variables": var}).values for var in varnames_tar}]
            if lt_embed:
                time_embeds = get_date_embeds(pd.to_datetime(time))
                all_streams = all_streams + time_embeds 
            yield tuple(all_streams)
        
    def gen_unnamed(darr_in, darr_tar, lt_embed: bool = False):
        """
        Generator with unnamed output-streams
        """
        ntimes = len(darr_in["time"])
        for t in range(ntimes):
            time = darr_in.isel({"time": t})["time"].values
            all_streams = [darr_in.isel({"time": t}).values, darr_tar.isel({"time": t}).values]
            if lt_embed:
                time_embeds = get_date_embeds(pd.to_datetime(time))
                all_streams = all_streams + time_embeds 

            yield tuple(all_streams)

    def get_date_embeds(dt_obj):
        """
        Create one-hot-encoding for month and hour of the day
        """
        hour, month = dt_obj.hour, dt_obj.month - 1
        date_embed = [OneHotEncoder(month, 12), OneHotEncoder(hour, 24)]

        return date_embed

    if named_targets is True:
        gen_now = gen_named
    else:
        gen_now = gen_unnamed
        
    # create output signatures from first sample
    s0 = next(iter(gen_now(da_in, da_tar, lembed_date)))
    sample_spec_in = tf.TensorSpec(s0[0].shape, dtype=s0[0].dtype)
    if named_targets is True:
        sample_spec_tar = {var: tf.TensorSpec(s0[1][var].shape, dtype=s0[1][var].dtype) for var in varnames_tar}
    else:
        sample_spec_tar = tf.TensorSpec(s0[1].shape, dtype=s0[1].dtype)

    # re-instantiate the generator and build TF dataset
    gen_train = gen_now(da_in, da_tar, lembed_date)

    if lembed_date is True:
        sample_spec_embed = [tf.TensorSpec(s0[i].shape, dtype=s0[i].dtype) for i in np.arange(2,4)]
        sample_spec_all = [sample_spec_in, sample_spec_tar] + sample_spec_embed

    else:
        sample_spec_all =  [sample_spec_in, sample_spec_tar]

    data_iter = tf.data.Dataset.from_generator(lambda: gen_train,
                                               output_signature=tuple(sample_spec_all))
            
    return data_iter

In [119]:
dsf511 = make_tf_dataset(da, lembed_date=True, named_targets=True)

print(dsf)

<FlatMapDataset shapes: ((96, 120, 9), {t_2m_tar: (96, 120), hsurf_tar: (96, 120)}, (12,), (24,)), types: (tf.float64, {t_2m_tar: tf.float64, hsurf_tar: tf.float64}, tf.float32, tf.float32)>
